In [1]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, 
)

# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action

# from googletrans import Translator
# import translators as ts

import re, math, random, json
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from tqdm import tqdm

from module.preprocess import preprocess

# import demoji

# from cleantext import clean
# from pykospacing import Spacing
# from hanspell import spell_checker

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# train_json = './dataset//nikluge-sa-2022-train.jsonl'
# train_raw = pd.read_json(train_json, lines=True)
# train_raw.annotation = train_raw.annotation.apply(lambda x: str(x))
# train_raw = train_raw[['sentence_form', 'annotation']]
# dup = train_raw.duplicated(keep=False)
# train_raw[dup]

In [3]:
# dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
# dev_raw = pd.read_json(dev_json, lines=True)
# dev_raw.annotation = dev_raw.annotation.apply(lambda x: str(x))
# dev_raw = dev_raw[['sentence_form', 'annotation']]
# dup = dev_raw.duplicated(keep=False)
# dev_raw[dup]

In [4]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

In [5]:
train = pd.read_json(train_json, lines=True)
dev = pd.read_json(dev_json, lines=True)
test = pd.read_json(test_json, lines=True)

In [6]:
# train.iloc[2319]
train = train.drop(2319)
# dev.iloc[1692]
dev = dev.drop(1692)

# Preprocess

## Cleansing

### Before

In [7]:
for el in train.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False).sentence_form:
    print(el)

무더운 여름 에어컨 틀어놓고 고 앞에 마루를 틀어주면 시원한 공기를 순환시켜 실내를 골고루 시원하게 해줘요~
수분 영양 쏙쏙~
체크무늬랑 빨강초록 예쁘네ㅋ
지친 피부에 풍부하 수분을 수시로 공급해주고 촉촉하고 윤기나게~
흘러내릴 듯 말듯 푸딩같은 신기한 크림 #퓨어마인드로맨틱로즈크림
내가 애정하는 #슈에무라
피부 곳곳을 적시듯 스며드는 텍스처 대박임
먹을 수 있는 원료로 사용한만큼.. 걱정없이 사용해볼 수 있겠어유..
소금샴푸라니 완전 신기해😃
전용 받침대의 경사면으로 물빠짐과 위생적인 관리가 가능해 이점도 참 좋네요^^
역할놀이를 하면서 공감능력 향상되고 아이 사회성이 좋아진다길래 갖고 싶었던 맥포머스
이거 사용 후 #마스크 착용 필요도 없고.. #봄 꽃가루가 심해서 외출시 불편했는데.. 이젠 걱정 노놉~
효과대박쓰
앰플은 여행용으로 딱인거 같아요 💛
근데 터치가 적응이 안 돼.


In [8]:
train.sentence_form = train.sentence_form.apply(preprocess)
dev.sentence_form = dev.sentence_form.apply(preprocess)
test.sentence_form = test.sentence_form.apply(preprocess)
total = pd.concat([train, dev, test])

Test

In [9]:
case = total.sentence_form.str.contains('r[^A-Za-z0-9가-힣\s]+', case=False, flags=0, na=None, regex=True)
for e in total[case].sentence_form:
    print(e)

### After

In [10]:
for i, row in total[['id', 'sentence_form']].sample(n=25, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False).iterrows():
    print(row.id, '\t', row.sentence_form)

nikluge-sa-2022-test-01303 	 핸들링 아너스 휴스톰
nikluge-sa-2022-dev-00922 	 좋아하는 향은 아니지만 친해져보쟈
nikluge-sa-2022-train-01255 	 꽃잎 리얼이니
nikluge-sa-2022-test-00338 	 한번도 사용해보지 않은 사람은 있어도한번만 사용한 사람은 없을정도로재구매율도 엄청나고 많은 사랑을받고있는 에센스죠
nikluge-sa-2022-train-02001 	 미백탄력주름수분공급 00시간 지속력까지 WOW
nikluge-sa-2022-dev-02034 	 요즘 핫한 스틸라 키튼카르마
nikluge-sa-2022-test-00325 	 따뜻한 핑크라서 차가운 느낌은 거의 없어요
nikluge-sa-2022-dev-00262 	 여행시 필수품 천연입욕제 아르비앙이에요
nikluge-sa-2022-train-02941 	 한 오분 정도가 지나면 흡수가 싹 되더라구요
nikluge-sa-2022-test-01715 	 0000MAH 배터리로 충전 후 오랜 시간 사용가능하구요
nikluge-sa-2022-dev-00240 	 요즘 넘 애정하는 슈에무라X슈퍼마리오 컬렉션
nikluge-sa-2022-test-00236 	 풍성해서 조금만 짜도 잘 분포되고 발림성도 좋아요
nikluge-sa-2022-dev-00718 	 아침에 엉덩이가 보송하니깐 일어나는 것도 상쾌한가봐요
nikluge-sa-2022-test-01954 	 크기도 꽤 넉넉해요
nikluge-sa-2022-train-02091 	 순하고 사용하기 굿
nikluge-sa-2022-dev-01372 	 그 어떤 빈틈도 뭉침도 허락하지 않고 지속력 좋아 요즘 매일 사용하는 데일리쿠션
nikluge-sa-2022-train-00668 	 오직 보습제들로만 이루어진 고농축 고보습 앰플
nikluge-sa-2022-dev-01658 	 좌우로 움직이는 헤드도 편하고 삼성리튬이온배터리라 한번 충전으로 00분까지 사용가능 하니 청소 하다가

In [11]:
total['check'] = total.sentence_form.str.find('OO')
for row in total[total.check > -1].sentence_form:
    print(row)

OO이유식 들어가면서 만들어줘야겠다 생각했는데 역시나 넘나 간편한 베이비쿡솔로
OO때부터 애정하며 사용중이지
OO칫솔 로 핫한 쿨샤전동칫솔 로 바꿨어요
OO가 사용할거라서 가성비 좋은 놈으로 겟
환절기라 메이크업을 해도 늘 좀 칙칙한 느낌이 있었는데 요즘은 화사한 OO랄까
패딩을 입어도 빛이나는 건 OO씨가 예뻐서 그런건가 패딩이 예뻐서 그런건가
OO씨가 이날 장착한 패딩과 신발은 모두 오니즈카타이거 제품이었는데요
거울보니까 내가봐도 예쁜데 OO도 자꾸 예쁘다해주니까 좋다
디올선미레드 라는 애칭이 벌써 붙었을만큼 핫한 000레드 컬러는 OO씨도 사용했네욤
OO추천템
좋아하는 뷰티유튜버 OO가 만든 쿠션제품이라고 해서 어떨지 넘나 궁금했는데 직접 써보니 대박 촉촉하고 컬러감 대박
OO의 애정템 이라하여 써본것이 벌써 몇 개째 쓰고있는지
이젠 고민말고 OO 너도 하나 겟하는 건 어때
푸석푸석 손상이심한 모발때문에 고민이많았는데 친구 추천으로 써본 헤드스파0 OO씨도 쓴다던데 역시 너무 좋네요
워낙 그립감이 좋아 OO가 써보더니 엄마는 손도 못대게 하더군요
출입이 자유로운 지퍼디자인 덕분에 OO가 답답해하지 않구 좋아하네요
친숙한 캐릭터라 OO가 좋아하구 다용도 끈이 있어 어디든 걸수있다는 장점
디자인도 OO한테 잘어울려서 올여름내내 입히겠어요
OO랑 놀러갈라믄 챙겨야될것들이 느무 많아서 고민이었는데 더블보냉가방 하나믄 끝나겠어요
OO와 함께하는 나들이 에는 보냉백 과 쇼퍼백 을 한번에 해결되는 마들린 더블 보온보냉백
OO가 보자마자 좋아해서 엄마는 그것만 봐도 행복했어요
조금만 더워도 땀흘리는 OO에게 완전 딱 맞는 실내복이네요
자연주의 0배 고농축이라 적은양으로도 세탁효과 대박이고 유해물질을 사용하지않아 안전하니 OO전용으로 세탁할때 사용중이에요
트렌드위드미에서 OO가 사용해 인싸템이 되어버린 톤업썬크림
OO향수로도 유명한 일본 향수 브랜드
디자인도 예쁘고 뚜껑은 OO가 갖고 놀기도 하는중
활동적인 OO에게 편하게 입힐수있어 좋으다
OO 애정템이라 참 좋아해서 

## Reformat

In [12]:
### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
]

more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

In [13]:
len(entity_property_pair)

25

In [14]:
def reformat(df):
    ep =[]
    p = []
    for index, row in df.iterrows():
        utterance = row.sentence_form
        id = row.id

        for pair in entity_property_pair:
            isPairInOpinion = False
            if pd.isna(utterance):
                break
            for annotation in row.annotation:
                entity_property = annotation[0]
                sentiment = annotation[2]
                if entity_property not in entity_property_pair:
                    print(row.id, entity_property)
                    break
                if sentiment not in polarity_id_to_name:
                    print(row.id, entity_property, sentiment)
                    break
                if entity_property == pair:
                    
                    true = [id, utterance, entity_property, tf_name_to_id['True']]
                    false = [id, utterance, entity_property, tf_name_to_id['False']]
                    if false in ep:
                        pop_id = ep.index(false)
                        ep.pop(pop_id)
                    ep.append(true)
                    p.append([id, utterance, entity_property, sentiment])
                    
                    
                    # ep.append([id, utterance, entity_property, tf_name_to_id['True']])
                    # p.append([id, utterance, entity_property, sentiment])
                    isPairInOpinion = True
                    break

            if isPairInOpinion is False:
                ep.append([id, utterance, pair, tf_name_to_id['False']])

    return ep, p

In [15]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [16]:
ep_train, p_train = reformat(train)
ep_train = pd.DataFrame(ep_train, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_train = pd.DataFrame(p_train, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

ep_dev, p_dev = reformat(dev)
ep_dev = pd.DataFrame(ep_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])
p_dev = pd.DataFrame(p_dev, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])

In [17]:
ep_total = pd.concat([ep_train, ep_dev])
p_total = pd.concat([p_train, p_dev])
len(ep_total), len(p_total)

(144825, 6198)

In [29]:
search = p_total[p_total.sentence_form.duplicated(keep=False) == True].sort_values('sentence_form').sentence_form
search.

2445                      0 무기자외선차단 성분으로 더욱 순하게 향도 순하니 좋다
2446                      0 무기자외선차단 성분으로 더욱 순하게 향도 순하니 좋다
2721    00000원이란 엄청난 가성비에 향까지 넘나 매력적인 니치향이라서 남친 0개주고 아...
2722    00000원이란 엄청난 가성비에 향까지 넘나 매력적인 니치향이라서 남친 0개주고 아...
1008        000ML 넉넉한 용량과 00가지 식물성오일로 저자극 프리미엄 베이비 워시 인데요
                              ...                        
1807    홍화 추출물과 차가버섯 추출물이 함유되어 있어 칙칙한 피부에 윤기와 탄력을 채워주는...
2793    화장솜에 톡톡 해주니깐 알갱이가 나오는데 전혀 피부에 부담주지 않고 닦아내기만 하면...
2794    화장솜에 톡톡 해주니깐 알갱이가 나오는데 전혀 피부에 부담주지 않고 닦아내기만 하면...
1541           휴대가 좋은 작은 사이즈라 좋은 토일렛퍼퓸 뒷사람을 위한 작은 배려 시크릿푸
1540           휴대가 좋은 작은 사이즈라 좋은 토일렛퍼퓸 뒷사람을 위한 작은 배려 시크릿푸
Name: sentence_form, Length: 810, dtype: object

In [18]:
def reform(data):
    ep =[]
    p = []
    for row in data:
        utterance = row['sentence_form']
        id = row['id']
        entity_property = row['entity_property']
        sentiment = row['sentiment']

        for pair in entity_property_pair:
            if not utterance:
                print('utterance is False')
                break
            if entity_property not in entity_property_pair:
                print(id, entity_property)
                break
            if sentiment not in polarity_id_to_name:
                print(id, entity_property, sentiment)
                break
            if entity_property == pair:
                true = [id, utterance, entity_property, tf_name_to_id['True']]
                false = [id, utterance, entity_property, tf_name_to_id['False']]
                if false in ep:
                    pop_id = ep.index(false)
                    ep.pop(pop_id)
                ep.append(true)
                p.append([id, utterance, entity_property, sentiment])
            else:
                false = [id, utterance, pair, tf_name_to_id['False']]
                if false in ep:
                    continue
                ep.append(false)

    return ep, p

In [19]:
p_total = p_total.to_dict(orient='records')
len(p_total)

6198

In [20]:
ep_tota, p_tota = reform(p_total)
len(ep_tota), len(p_tota)

In [22]:
pd.DataFrame(ep_tota).drop_duplicates()

,0,1,2,3
0,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#가격,1
1,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#다양성,1
2,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#디자인,1
3,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#인지도,1
4,nikluge-sa-2022-train-00001,둘쨋날은 미친듯이 밟아봤더니 기어가 헛돌면서 틱틱 소리가 나서 경악,본품#일반,1
...,...,...,...,...
145225,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,패키지/구성품#다양성,1
145226,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,패키지/구성품#디자인,1
145227,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,패키지/구성품#일반,1
145228,nikluge-sa-2022-dev-02794,안경 쓰고 숨 푸악푸악 내쉬어 봤는데 김서림도 거의 없음,패키지/구성품#편의성,1


In [ ]:
# for i, row in ep_total.sort_values(['id', 'labels']).iterrows():
#     print(row.id, '\t', row.labels)
#     if i ==10000:
#         break

In [ ]:
### more than 1 entitiy in a single row
# condition = train.annotation.apply(lambda x: len(x)) > 1
# print(train.annotation[condition].count())
# print(train.annotation[condition].apply(lambda x: len(x)).sum())
# 489-235

In [ ]:
p_binary_train = reformat_p_binary(p_train)
p_binary_train = pd.DataFrame(p_binary_train, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_train

In [ ]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_dev

## Counting

In [ ]:
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))
ep_train = ep_train.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_train = p_train.drop_duplicates()
p_dev = p_dev.drop_duplicates()
p_binary_train = p_binary_train.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
print('\nafter drop_duplicates')
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))

## Save Files

In [ ]:
save_path = './dataset/cleaned_v1'

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)
p_train.to_csv(f'{save_path}/pc_train.csv', index=False)
p_dev.to_csv(f'{save_path}/pc_dev.csv', index=False)
p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

# ASC Augmentation

In [ ]:
model_checkpoint = '/content/drive/MyDrive/aspect_based_sentiment_analysis/base_model/klue_roberta_base/v2/klue_roberta_base_mlm/checkpoint-19860'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
sTokens = tokenizer.all_special_tokens

def delTokens(sent):
    sent = sent.split(' ')
    temp = []
    for e in sent:
        if e not in sTokens:
            temp.append(e)
    return ' '.join(temp)

In [ ]:
positive, negative, neutral = p_train[p_train.sentiment == 'positive'], p_train[p_train.sentiment == 'negative'], p_train[p_train.sentiment == 'neutral']

In [ ]:
len(positive), len(negative), len(neutral)

In [ ]:
(58 * 3) * 4 * 3, (95 * 3) * 4 * 2 # bt ri rr

Back Translation / Random Insertion / Random Replacement / Random Swap / Random Deletion

In [ ]:
def backTrans(text):
    aug1 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='en')
    aug1 = ts.papago(aug1, sleep_seconds=5, from_language='en', to_language='ko')

    aug2 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='ja')
    aug2 = ts.papago(aug2, sleep_seconds=5, from_language='ja', to_language='ko')

    return [aug1, aug2]

def randomInsert(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomReplace(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomSwap(num, sample):
    aug = naw.RandomWordAug(action='swap', aug_min=1, aug_max=1, aug_p=0.3)    
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

def randomSplit(num, sample):
    aug = naw.SplitAug(aug_min=1, aug_max=1, aug_p=0.3, min_char=3)
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

In [ ]:
(58 * 3) * 5 * 4, (95 * 3) * 4 * 3 # bt ri rr

In [ ]:
def backtransRoutine(data2augment, output_path):
    print('back translation started.')
    temp = []
    for row in data2augment:
        augs = backTrans(row[1])
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(len(f'back translation finished.\ncurrent count: {len(data2augment)}'))

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
import os

def edaRoutine(data2augment, ri, rr, output_path):
    print(f'current count: {len(data2augment)}')
    print('random insertion started.')
    temp = []
    for row in data2augment:
        augs = randomInsert(ri, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random insertion finished.\ncurrent count: {len(data2augment)}')

    print('random replacement started.')
    temp = []
    for row in data2augment:
        augs = randomReplace(rr, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random replacement finished.\ncurrent count: {len(data2augment)}')

    print('random swap and split started.')
    while len(data2augment) < len(positive):
        temp = []
        k = random.randrange(len(negative))
        id, text, entity, sentiment = data2augment[k]

        selector = random.randint(0,1)
        if selector == 0:
            augs = randomSwap(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        else:
            augs = randomSplit(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        if len(data2augment)%25 == 0:
            print(f'random swap and split in progress.\ncurrent count: {len(data2augment)}')

    print(f'whole augmentation routine finished.\ntotal count: {len(data2augment)}')

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
### negative
# # back translation

# data2augment = negative.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_bt.csv'
output_path = os.path.join(output_folder, output_file)

# negative_bt = backtransRoutine(data2augment, output_path)
negative_bt = pd.read_csv(output_path)
negative_bt = negative_bt.values.tolist()
# RI / RR

ri = 4 # times - 1
rr = 3 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_aug.csv'
output_path = os.path.join(output_folder, output_file)

# negative_aug = edaRoutine(negative_bt, ri, rr, output_path)
negative_aug = pd.read_csv(output_path)

In [ ]:
negative_aug
negative_aug = negative_aug.drop_duplicates()

In [ ]:
# negative_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# negative_aug.sort_values('id').head(50).sentence_form

In [ ]:
### neutral
# back translation

# data2augment = neutral.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_bt.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_bt = backtransRoutine(data2augment, output_path)
neutral_bt = pd.read_csv(output_path)
neutral_bt = neutral_bt.values.tolist()

# RI / RR

ri = 3 # times - 1
rr = 2 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_aug.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_aug = edaRoutine(neutral_bt, ri, rr, output_path)
neutral_aug = pd.read_csv(output_path)

In [ ]:
neutral_aug
neutral_aug = neutral_aug.drop_duplicates()

In [ ]:
# neutral_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# neutral_aug.sort_values('id').head(50).sentence_form

In [ ]:
p_train_aug = pd.concat([positive, negative_aug, neutral_aug])

In [ ]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [ ]:
p_binary_train_aug = reformat_p_binary(p_train_aug)
p_binary_train_aug = pd.DataFrame(p_binary_train_aug, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_train_aug

In [ ]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_dev

# Counting

In [ ]:
len(ep_train), len(ep_dev), len(p_train), len(p_dev)

In [ ]:
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

In [ ]:
ep_train = ep_train.drop_duplicates()
p_binary_train_aug = p_binary_train_aug.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

# Export

In [ ]:
%cd /content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11

# train.to_csv('raw_train.csv', index=False)
# dev.to_csv('raw_dev.csv', index=False)
# test.to_csv('raw_test.csv', index=False)

ep_train.to_csv('ce_train.csv', index=False)
p_binary_train_aug.to_csv('pc_binary_train_aug.csv', index=False)
ep_dev.to_csv('ce_dev.csv', index=False)
p_binary_dev.to_csv('pc_binary_dev.csv', index=False)

In [ ]:
# emojis = pd.concat([ep_train.sentence_form, p_train.sentence_form, ep_dev.sentence_form, p_dev.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
# emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))

In [ ]:
df = pd.read_csv('ce_train.csv')
df[df.id == 'nikluge-sa-2022-train-00065']

In [ ]:
df = pd.read_csv('ce_dev.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_train_aug.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_dev.csv')
df